<h1 style="color: #FF5733; font-size: 56px;"> Cyber Bullying 1.3.2.	SQL script for database and wrangling</h1>

The password for rds is hidden to maintain data  privacy

## 🧾 Inspect All Tables and Schemas in MySQL RDS (Shieldspace)

This block connects to the `shieldspace` database on AWS RDS using `mysql.connector` and retrieves a list of all tables in the database. For each table, it runs the `DESCRIBE` command to print the column names and their corresponding data types in a readable format.

It’s a useful way to audit your database schema at a glance, verify table structures, and confirm the success of previous data uploads or transformations.

In [ ]:
import mysql.connector

# ─── Database Connection Info ─────────────────────────────
username = "admin"
password = "************"
host     = "shieldspace.ck1sygqoe3hd.us-east-1.rds.amazonaws.com"
database = "shieldspace"

# ─── Connect to the Database ──────────────────────────────
try:
    conn = mysql.connector.connect(
        user=username,
        password=password,
        host=host,
        database=database
    )

    cursor = conn.cursor()

    # ─── Fetch all table names ─────────────────────────────
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()

    print(f"\n📋 Total Tables in '{database}': {len(tables)}\n")

    # ─── Loop through tables and describe each ─────────────
    for (table_name,) in tables:
        print(f"\n🔹 Table: `{table_name}`")
        cursor.execute(f"DESCRIBE `{table_name}`;")
        columns = cursor.fetchall()
        print("┌────────────┬──────────────┐")
        print("│ Column     │ Data Type    │")
        print("├────────────┼──────────────┤")
        for column in columns:
            print(f"│ {column[0]:<10} │ {column[1]:<12} │")
        print("└────────────┴──────────────┘")

except mysql.connector.Error as err:
    print(f"❌ Error: {err}")
finally:
    if conn.is_connected():
        cursor.close()
        conn.close()

## 🗄️ RDS Table Creation and Setup (PostgreSQL)

In this step, we connected our Jupyter Notebook to an AWS-hosted PostgreSQL database using SQLAlchemy and created two structured tables: `cb_multi_labeled_balanced` and `youtube_parsed_dataset_sentiment`. We then renamed the first table to `bully_message` and added an auto-incrementing primary key column `id`. Finally, we verified the connection and schema by querying and previewing the top 10 rows from both tables, ensuring the setup is ready for further data insertion and analysis.

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Text, Table, MetaData
import pandas as pd

# ------------------ Step 1: Connect to RDS PostgreSQL ------------------
username = 'admin'
password = '**********'
host = 'shieldspace.ck1sygqoe3hd.us-east-1.rds.amazonaws.com'
database = 'shieldspace'

# Format connection string
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:5432/{database}")
metadata = MetaData()

# ------------------ Step 2: Define Tables ------------------

# Table 1: cb_multi_labeled_balanced (will be renamed to bully_message)
cb_multi_labeled_balanced = Table("cb_multi_labeled_balanced", metadata,
    Column("text", Text),
    Column("label", String(255))
)

# Table 2: youtube_parsed_dataset_sentiment
youtube_parsed_dataset_sentiment = Table("youtube_parsed_dataset_sentiment", metadata,
    Column("index", Integer),
    Column("UserIndex", String(255)),
    Column("Text", Text),
    Column("Number_of_Comments", Integer),
    Column("Number_of_Subscribers", Integer),
    Column("Membership_Duration", Integer),
    Column("Number_of_Uploads", Integer),
    Column("Profanity_in_UserID", Integer),
    Column("Age", Integer),
    Column("oh_label", Integer),
    Column("sentiment_class", String(255)),
    Column("emotion_scores", Text)
)

# ------------------ Step 3: Create Tables on RDS ------------------
metadata.create_all(engine)

# ------------------ Step 4: Rename and Add Primary Key ------------------
with engine.connect() as conn:
    # Rename the table
    conn.execute("ALTER TABLE cb_multi_labeled_balanced RENAME TO bully_message;")
    
    # Add a serial primary key column
    conn.execute("ALTER TABLE bully_message ADD COLUMN id SERIAL PRIMARY KEY;")

# ------------------ Step 5: Preview Sample Rows ------------------

# Fetch top 10 rows from each table (if any data is loaded)
try:
    bully_df = pd.read_sql("SELECT * FROM bully_message LIMIT 10;", engine)
    print("👁️‍🗨️ Bully Message Table Preview:\n", bully_df)
except Exception as e:
    print("⚠️ Error fetching from bully_message:", e)

try:
    youtube_df = pd.read_sql("SELECT * FROM youtube_parsed_dataset_sentiment LIMIT 10;", engine)
    print("🎥 YouTube Dataset Preview:\n", youtube_df)
except Exception as e:
    print("⚠️ Error fetching from youtube_parsed_dataset_sentiment:", e)

$$
\textcolor{purple}{\textbf{\small \text{Database management and visulization – Iteration 2}}}
$$

## 📊 Race-wise Hate Speech Summary Upload to MySQL (RDS)

In this step, we connected to an AWS-hosted MySQL RDS instance and processed the `final_hateXplain.csv` dataset to summarize hate/offensive messages by race category. We filtered out neutral entries and excluded "no_race" labels. A summary DataFrame with counts and percentages was created and uploaded to a new MySQL table named `race_summary`. Finally, we queried the table back to confirm successful upload and inspect the data stored in the cloud database.

In [ ]:
# ─── Packages ─────────────────────────────────────────────────────────────
import pandas as pd
from sqlalchemy import create_engine, text
import sqlalchemy.types as satypes

# # ─── 1.  CONNECTION INFO  ─────────────────────────────────────────────────
# username = "admin"               
# password = "*************"     
# host     = "shieldspace.ck1sygqoe3hd.us-east-1.rds.amazonaws.com"
# database = "shieldspace"

# engine = create_engine(
#     f"mysql+mysqlconnector://{username}:{password}@{host}/{database}",
#     pool_pre_ping=True,            # quick dead-connection check
# )

# # ─── 2.  TEST THE CONNECTION  ─────────────────────────────────────────────
# try:
#     with engine.connect() as conn:
#         conn.execute(text("SELECT 1"))
#     print("✅  Successfully connected to the database.")
# except Exception as e:
#     raise RuntimeError(f"❌  Database connection failed: {e}")

# ─── 3.  BUILD THE SUMMARY DATAFRAME  ─────────────────────────────────────
df = (
    pd.read_csv("final_hateXplain.csv")
      .query("label != 'normal'")                           # keep hate/offensive
)
df = df[~df["Race"].str.lower().str.replace(" ", "_").eq("no_race")]

summary = (
    df["Race"]
      .value_counts()
      .rename_axis("race")
      .reset_index(name="count")
      .assign(percentage=lambda d: d["count"] / d["count"].sum())
)

total_row = pd.DataFrame({
    "race": ["Total"],
    "count": [summary["count"].sum()],
    "percentage": [1.0]
})
summary = pd.concat([summary, total_row], ignore_index=True)

# ─── 4.  PUSH TO MYSQL  ───────────────────────────────────────────────────
table_name = "race_summary"        # change if you like

dtype_map = {
    "race": satypes.VARCHAR(32),
    "count": satypes.INTEGER(),
    "percentage": satypes.FLOAT(asdecimal=False),
}

summary.to_sql(
    name       = table_name,
    con        = engine,
    if_exists  = "replace",        # "append" if you want to keep old rows
    index      = False,
    dtype      = dtype_map,
)

print(f"✅  Table `{table_name}` written to `{database}` database.")

# ─── 5.  OPTIONAL: QUERY BACK TO VERIFY  ──────────────────────────────────
with engine.connect() as conn:
    check_df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
print("\nPreview from DB:")
print(check_df)

## 🛠️ Full Pipeline: CSV Summaries → MySQL RDS Upload and Verification

In this pipeline, we connected to an AWS-hosted MySQL RDS instance and uploaded structured datasets with analysis summaries. We first processed the `Cyberbullying_Words_with_Coping_Tips.csv` file and created a `cyberbullying_terms` table. Then, we generated an age-wise distribution of aggressive comments from the `youtube_parsed_dataset_sentiment.csv` and uploaded it as `aggressive_age_distribution`. We also analyzed the `final_hateXplain.csv` dataset to produce a `race_summary` (excluding "no_race") and a religion-wise hate/offense breakdown (`religion_summary_by_label`), each with count and percentage columns. All tables were uploaded to MySQL and verified by querying and previewing the top rows.

In [ ]:
# ─── Imports ─────────────────────────────────────────────────────────────
import pandas as pd
from sqlalchemy import create_engine, text
import sqlalchemy.types as satypes

# ─── 1. MySQL RDS Connection ─────────────────────────────────────────────
username = "admin"
password = "******************"
host     = "shieldspace.ck1sygqoe3hd.us-east-1.rds.amazonaws.com"
database = "shieldspace"

engine = create_engine(
    f"mysql+mysqlconnector://{username}:{password}@{host}/{database}",
    pool_pre_ping=True,
)

try:
    with engine.connect() as conn:
        conn.execute(text("SELECT 1"))
    print("✅ Connected to RDS MySQL successfully.")
except Exception as e:
    raise RuntimeError(f"❌ Connection failed: {e}")

# ─── 2. Cyberbullying Words Upload ───────────────────────────────────────
print("\n▶ Uploading cyberbullying_terms")
df_cb = pd.read_csv("Cyberbullying_Words_with_Coping_Tips.csv")
cb_sql_types = {
    "word": satypes.VARCHAR(50),
    "frequency": satypes.INTEGER(),
    "coping_tip": satypes.TEXT()
}
df_cb.to_sql("cyberbullying_terms", con=engine, if_exists="replace", index=False, dtype=cb_sql_types)
print("✅ Table `cyberbullying_terms` uploaded.\n")

# ─── 3. Aggressive Age Distribution Summary ──────────────────────────────
print("▶ Creating aggressive_age_distribution summary")
df_youtube = pd.read_csv("youtube_parsed_dataset_sentiment.csv")
df_aggr = df_youtube[df_youtube["sentiment_class"] == "Aggressive"].copy()
df_aggr["Age"] = pd.to_numeric(df_aggr["Age"], errors="coerce")
df_aggr = df_aggr.dropna(subset=["Age"])

age_summary = (
    df_aggr["Age"]
    .value_counts()
    .rename_axis("age")
    .reset_index(name="count")
    .sort_values("age")
    .astype({"age": "int", "count": "int"})
)

age_sql_types = {
    "age": satypes.INTEGER(),
    "count": satypes.INTEGER(),
}

age_summary.to_sql("aggressive_age_distribution", con=engine, if_exists="replace", index=False, dtype=age_sql_types)
print("✅ Table `aggressive_age_distribution` uploaded.")

# Preview
with engine.connect() as conn:
    preview_age = pd.read_sql("SELECT * FROM aggressive_age_distribution ORDER BY age LIMIT 10", conn)
print("\n📊 Preview: aggressive_age_distribution")
print(preview_age.to_string(index=False))

# ─── 4. Race Summary from HateXplain ─────────────────────────────────────
print("\n▶ Creating race_summary")
df_race = pd.read_csv("final_hateXplain.csv").query("label != 'normal'")
df_race = df_race[~df_race["Race"].str.lower().str.replace(" ", "_").eq("no_race")]

race_summary = (
    df_race["Race"]
    .value_counts()
    .rename_axis("race")
    .reset_index(name="count")
    .assign(percentage=lambda d: d["count"] / d["count"].sum())
)

total_row = pd.DataFrame({
    "race": ["Total"],
    "count": [race_summary["count"].sum()],
    "percentage": [1.0]
})
race_summary = pd.concat([race_summary, total_row], ignore_index=True)

race_sql_types = {
    "race": satypes.VARCHAR(32),
    "count": satypes.INTEGER(),
    "percentage": satypes.FLOAT(asdecimal=False),
}
race_summary.to_sql("race_summary", con=engine, if_exists="replace", index=False, dtype=race_sql_types)
print("✅ Table `race_summary` uploaded.")

# ─── 5. Religion vs Label Summary ────────────────────────────────────────
print("\n▶ Creating religion_summary_by_label")
df_religion = pd.read_csv("final_hateXplain.csv")
df_religion = df_religion[df_religion["label"] != "normal"].copy()

drop_tokens = {"no_religion", "non_religious", "nonreligious", "none"}
df_religion = df_religion[~df_religion["Religion"].str.lower().str.replace(" ", "_").isin(drop_tokens)]

religion_summary = (
    df_religion.groupby(["Religion", "label"], as_index=False)
    .size()
    .rename(columns={"size": "count"})
)
religion_summary["pct_within_religion"] = (
    religion_summary["count"] /
    religion_summary.groupby("Religion")["count"].transform("sum")
)

religion_sql_types = {
    "Religion": satypes.VARCHAR(48),
    "label": satypes.VARCHAR(12),
    "count": satypes.INTEGER(),
    "pct_within_religion": satypes.FLOAT(asdecimal=False),
}
religion_summary.to_sql("religion_summary_by_label", con=engine, if_exists="replace", index=False, dtype=religion_sql_types)
print("✅ Table `religion_summary_by_label` uploaded.")

# Preview
with engine.connect() as conn:
    preview_religion = pd.read_sql(
        "SELECT * FROM religion_summary_by_label ORDER BY Religion, label LIMIT 12",
        conn
    )
print("\n📊 Preview: religion_summary_by_label")
print(preview_religion.to_string(index=False, formatters={
    "count": "{:,}".format,
    "pct_within_religion": "{:.4f}".format,
}))

## 🧾 Bullying Data Summaries – Miscellaneous & Sexual Orientation (MySQL Upload)

This notebook block connects to the AWS MySQL RDS database and processes the `final_hateXplain.csv` dataset to extract insights specifically related to bullying. It filters out only offensive and hate speech rows, then removes placeholder or unknown values from the `Miscellaneous` and `Sexual Orientation` columns.

Two summaries are generated:
- `misc_by_label`: a count of bullying messages grouped by `Miscellaneous` category and `label`.
- `so_bullying_totals`: total bullying messages grouped by sexual orientation.

Both tables are uploaded to the MySQL database, and a preview of their contents is printed for verification.

In [ ]:
############################################################################
# 0.  IMPORTS & DB CONNECTION
############################################################################
import pandas as pd
from sqlalchemy import create_engine, text
import sqlalchemy.types as satypes

username = "admin"
password = "fit5120ta30"
host     = "shieldspace.ck1sygqoe3hd.us-east-1.rds.amazonaws.com"
database = "shieldspace"

engine = create_engine(
    f"mysql+mysqlconnector://{username}:{password}@{host}/{database}",
    pool_pre_ping=True,
)

# quick connection test
with engine.connect() as conn:
    conn.execute(text("SELECT 1"))
print("✅  Connected to RDS")

############################################################################
# 1.  LOAD & FILTER DATA  (bullying only, drop placeholders)
############################################################################
df = pd.read_csv("final_hateXplain.csv")
df = df[df["label"].isin(["offensive", "hatespeech"])].copy()     # bullying rows

misc_drop = {"none", "na", "n_a", "no_misc", "nomisc", "other", ""}
so_drop   = {"none", "na", "n_a", "unknown", "no_orientation", "other", ""}

df = df[~df["Miscellaneous"].str.lower().str.replace(" ", "_").isin(misc_drop)]
df = df[~df["Sexual Orientation"].str.lower().str.replace(" ", "_").isin(so_drop)]

############################################################################
# 2-A.  BUILD  Miscellaneous × label  SUMMARY
############################################################################
misc_summary = (
    df.groupby(["Miscellaneous", "label"], as_index=False)
      .size()
      .rename(columns={
          "Miscellaneous": "category",
          "size"         : "count"
      })
)

############################################################################
# 2-B.  BUILD  Sexual-orientation bullying totals
############################################################################
so_summary = (
    df["Sexual Orientation"]
      .value_counts()
      .rename_axis("orientation")
      .reset_index(name="count")
)

############################################################################
# 3.  WRITE BOTH TABLES TO MYSQL
############################################################################
# dtype mappings
misc_dtypes = {
    "category": satypes.VARCHAR(64),
    "label"   : satypes.VARCHAR(12),
    "count"   : satypes.INTEGER()
}

so_dtypes = {
    "orientation": satypes.VARCHAR(32),
    "count"      : satypes.INTEGER()
}

misc_summary.to_sql(
    name      ="misc_by_label",
    con       =engine,
    if_exists ="replace",          # "append" if you prefer
    index     =False,
    dtype     =misc_dtypes,
)
so_summary.to_sql(
    name      ="so_bullying_totals",
    con       =engine,
    if_exists ="replace",
    index     =False,
    dtype     =so_dtypes,
)

print("✅  Tables `misc_by_label` and `so_bullying_totals` written to MySQL")

############################################################################
# 4.  VERIFY  (read back a few rows)
############################################################################
with engine.connect() as conn:
    print("\nPreview misc_by_label:")
    print(pd.read_sql("SELECT * FROM misc_by_label LIMIT 8", conn).to_string(index=False))

    print("\nPreview so_bullying_totals:")
    print(pd.read_sql("SELECT * FROM so_bullying_totals", conn).to_string(index=False))

## 📊 Full Visual Analytics Suite – Cyberbullying & Hate Speech Insights from MySQL

In this notebook cell, we connected directly to our AWS MySQL RDS database and visualized structured data stored across several curated tables. The suite includes:

- A **treemap** visualizing the distribution of hate/offensive content by race (`race_summary`).
- A **sunburst**, **grouped bar chart**, and **Altair dropdown bar chart** showing label distribution across religions (`religion_summary_by_label`).
- A **stacked bar chart** for miscellaneous bullying categories by label (`misc_by_label`).
- A **reversed line chart** representing bullying tweet counts across sexual orientations (`so_bullying_totals`).
- A **violin plot with swarm overlay** visualizing age distribution of users posting aggressive comments (`aggressive_age_distribution`).

This visual suite gives a comprehensive, interactive view of key bullying dimensions (race, religion, orientation, age) using live data from the MySQL cloud database.

In [ ]:
"""
FULL VISUAL SUITE  —  data pulled straight from MySQL
====================================================
Relies on these tables (created earlier):

  • race_summary                    (race, count)
  • religion_summary_by_label       (Religion, label, count)
  • misc_by_label                   (category, label, count)
  • so_bullying_totals              (orientation, count)
  • aggressive_age_distribution     (age, aggressive_comment_count)

Run in one notebook cell. Auto-installs any missing libs.
"""

# ────────────────────────────────────────────────────────────────────────
# 0. Imports (auto-install helper)
# ────────────────────────────────────────────────────────────────────────
import sys, subprocess, importlib, warnings

def ensure(pkg):
    try:
        return importlib.import_module(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
        return importlib.import_module(pkg)

pd   = ensure("pandas")
plt  = ensure("matplotlib.pyplot")
sns  = ensure("seaborn")
sq   = ensure("squarify")
px   = ensure("plotly.express")
alt  = ensure("altair")
np   = ensure("numpy")
from sqlalchemy import create_engine, text

plt.style.use("ggplot")
sns.set_palette("Set2")
warnings.filterwarnings("ignore", category=FutureWarning)

# ────────────────────────────────────────────────────────────────────────
# 1. DB connection
# ────────────────────────────────────────────────────────────────────────
username = "admin"
password = "fit5120ta30"
host     = "shieldspace.ck1sygqoe3hd.us-east-1.rds.amazonaws.com"
database = "**********"

engine = create_engine(
    f"mysql+mysqlconnector://{username}:{password}@{host}/{database}",
    pool_pre_ping=True,
)

with engine.connect() as conn:
    conn.execute(text("SELECT 1"))
print("✅  Connected to RDS")

# ════════════════════════════════════════════════════════════════════════
# A.  Race  —  Treemap
# ════════════════════════════════════════════════════════════════════════
race_df = pd.read_sql("SELECT race, count FROM race_summary", engine)

sizes  = race_df["count"].values
labels = [f"{r}\n{c:,} ({c/sizes.sum():.1%})" for r, c in zip(race_df["race"], sizes)]
colors = sns.color_palette("Set3", n_colors=len(race_df))

plt.figure(figsize=(10,6))
sq.plot(sizes=sizes, label=labels, color=colors,
        pad=True, bar_kwargs=dict(edgecolor="white", linewidth=2))
plt.axis("off")
plt.title("Race distribution (hatespeech + offensive)", fontsize=16, weight="bold")
plt.tight_layout(); plt.show()

# ════════════════════════════════════════════════════════════════════════
# B.  Religion  —  Sunburst  +  Grouped bar  +  Altair dropdown
# ════════════════════════════════════════════════════════════════════════
rel_df = pd.read_sql("SELECT Religion, label, count FROM religion_summary_by_label", engine)

# Sunburst (Plotly)
fig_sb = px.sunburst(
    rel_df, path=["label","Religion"], values="count",
    color="label", color_discrete_map={"offensive":"#FF6361","hatespeech":"#003F5C"},
    hover_data={"count":":,"},
)
fig_sb.update_traces(insidetextorientation="radial")
fig_sb.update_layout(title="Label → Religion sunburst", margin=dict(t=40,l=0,r=0,b=0))
fig_sb.show()

# Grouped bar (static)
bar_df = rel_df.pivot(index="Religion", columns="label", values="count").fillna(0)
bar_df.plot(kind="bar", figsize=(10,5), color=sns.color_palette("Set2", 2))
plt.ylabel("Tweet count"); plt.title("Religion by label – grouped bars")
plt.xticks(rotation=45, ha="right"); plt.tight_layout(); plt.show()

# Altair dropdown (count ↔ percentage)  — new param API
alt.data_transformers.disable_max_rows()

abs_df = bar_df.reset_index().melt("Religion", var_name="label", value_name="val")
pct_df = abs_df.assign(
    val=lambda d: d.groupby("Religion")["val"].transform(lambda x: x/x.sum()),
    view="percentage"
)
abs_df["view"] = "count"
viz_df = pd.concat([abs_df, pct_df], ignore_index=True)

param_view = alt.param(name="view",
                       bind=alt.binding_select(options=["count","percentage"], name="Show "),
                       value="count")

alt.Chart(viz_df).add_params(param_view)\
    .transform_filter("datum.view === view")\
    .mark_bar()\
    .encode(
        x=alt.X("Religion:N", sort="-y"),
        y=alt.Y("val:Q", title="Tweets"),
        color=alt.Color("label:N", scale=alt.Scale(scheme="set2")),
        tooltip=["Religion:N","label:N","val:Q"]
    )\
    .properties(width=600, height=400,
                title="Religion – interactive view (non-religious removed)")\
    .display()

# ════════════════════════════════════════════════════════════════════════
# C.  Miscellaneous  —  Stacked bar
# ════════════════════════════════════════════════════════════════════════
misc_df = pd.read_sql("SELECT category, label, count FROM misc_by_label", engine)
misc_stack = misc_df.pivot(index="category", columns="label", values="count").fillna(0)
misc_stack.plot(kind="bar", stacked=True, figsize=(10,5))
plt.ylabel("Tweet count"); plt.title("Miscellaneous by label – stacked bar")
plt.xticks(rotation=45, ha="right"); plt.legend(title="Label", bbox_to_anchor=(1.02,1))
plt.tight_layout(); plt.show()

# ════════════════════════════════════════════════════════════════════════
# D.  Sexual Orientation  —  reversed line chart
# ════════════════════════════════════════════════════════════════════════
so_df = pd.read_sql("SELECT orientation, count FROM so_bullying_totals", engine)
so_df   = so_df.sort_values("count", ascending=False)
orient  = so_df["orientation"].iloc[::-1]
counts  = so_df["count"].iloc[::-1]

plt.figure(figsize=(9,5))
plt.plot(orient, counts, marker="o", linewidth=2)
plt.title("Bullying tweets by Sexual Orientation")
plt.xlabel("Sexual Orientation (reversed)"); plt.ylabel("Tweet count")
plt.xticks(rotation=45, ha="right"); plt.tight_layout(); plt.show()

# ════════════════════════════════════════════════════════════════════════
# E.  Age distribution  — Violin + Swarm
# ════════════════════════════════════════════════════════════════════════
age_df = pd.read_sql(
    "SELECT age, aggressive_comment_count AS n FROM aggressive_age_distribution", engine
)

ages_expanded = np.repeat(age_df["age"].values, age_df["n"].values)

plt.figure(figsize=(10,6))
sns.violinplot(x=ages_expanded, color="skyblue", inner="quartile")
sns.swarmplot(x=ages_expanded, color="darkblue", size=3, alpha=0.6)
plt.title("Age distribution for aggressive comments")
plt.xlabel("Age"); plt.ylabel("Density")
plt.tight_layout(); plt.show()

$$
\textcolor{purple}{\textbf{\small \text{Database management and visulization – Iteration 3}}}
$$

## 🔑 Add Primary Key to All MySQL Tables (if Missing)

This cell connects to the AWS-hosted `shieldspace` MySQL RDS database and iterates over every table. It checks whether each table already contains a primary key using the `information_schema` metadata.

If a table does **not** have a primary key, it adds a new column named `id` as an `INT NOT NULL AUTO_INCREMENT PRIMARY KEY`. If the table already has a primary key, it simply skips it with a confirmation message.

This ensures that all tables have a consistent primary key structure, which is essential for indexing, foreign key relationships, and safe data handling.

In [ ]:
import mysql.connector

# ─── DB Config ──────────────────────────────────────────────
config = {
    "user": "admin",
    "password": "***********",
    "host": "shieldspace.ck1sygqoe3hd.us-east-1.rds.amazonaws.com",
    "database": "shieldspace"
}

try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()

    # ─── Get All Tables ─────────────────────────────────────
    cursor.execute("SHOW TABLES;")
    tables = [table[0] for table in cursor.fetchall()]

    for table in tables:
        # Check if table already has a PRIMARY KEY
        cursor.execute(f"""
            SELECT COUNT(*) 
            FROM information_schema.table_constraints 
            WHERE table_schema = '{config["database"]}'
              AND table_name = '{table}'
              AND constraint_type = 'PRIMARY KEY';
        """)
        has_pk = cursor.fetchone()[0] > 0

        if not has_pk:
            print(f"⚙️ Adding 'id' column as PRIMARY KEY to `{table}`...")
            try:
                cursor.execute(f"""
                    ALTER TABLE `{table}`
                    ADD COLUMN id INT NOT NULL AUTO_INCREMENT PRIMARY KEY;
                """)
                conn.commit()
                print(f"✅ Done for `{table}`")
            except mysql.connector.Error as e:
                print(f"❌ Error modifying `{table}`: {e}")
        else:
            print(f"✔️ `{table}` already has a primary key.")

except mysql.connector.Error as err:
    print(f"❌ Connection error: {err}")
finally:
    if conn.is_connected():
        cursor.close()
        conn.close()

## 📤 Uploading Cyberbullying Statistics to MySQL RDS (Australia, 2024)

This cell connects to the `shieldspace` MySQL RDS database and uploads the `Australia_Cyberbullying_Stats_2024.csv` dataset into a structured table called `cyberbullying_statistics_2024`. The script:
- Strips and cleans column names.
- Assigns proper SQL data types (including float handling for percentages/counts).
- Replaces any existing table with the new data.
- Displays a preview of the first 10 rows after upload.

This step ensures that live, government-sourced statistics are accessible from the database for dashboard visualizations and further analysis.

In [ ]:
# ── 0. Imports & DB connection ────────────────────────────────────────────
import pandas as pd
from sqlalchemy import create_engine, text
import sqlalchemy.types as satypes

# RDS credentials
username = "admin"
password = "*************"
host     = "shieldspace.ck1sygqoe3hd.us-east-1.rds.amazonaws.com"
database = "shieldspace"

# MySQL connection
engine = create_engine(
    f"mysql+mysqlconnector://{username}:{password}@{host}/{database}",
    pool_pre_ping=True,
)

# Quick test
with engine.connect() as conn:
    conn.execute(text("SELECT 1"))
print("✅ Connected to RDS")

# ── 1. Load CSV ──────────────────────────────────────────────────────────
df = pd.read_csv("Australia_Cyberbullying_Stats_2024.csv")

# strip column names
df.columns = df.columns.str.strip()

# ── 2. Set Data Types ─────────────────────────────────────────────────────
dtype_map = {
    "Category"      : satypes.VARCHAR(64),
    "Statistic"     : satypes.TEXT(),
    "Value"         : satypes.FLOAT(asdecimal=False),  # allows both percent and count
    "Unit"          : satypes.VARCHAR(16),
    "Age Group"     : satypes.VARCHAR(32),
    "Source"        : satypes.TEXT(),
}

# ── 3. Upload to MySQL ────────────────────────────────────────────────────
df.to_sql(
    name="cyberbullying_statistics_2024",
    con=engine,
    if_exists="replace",     # change to "append" if needed
    index=False,
    dtype=dtype_map,
)

print("✅ Table `cyberbullying_statistics_2024` uploaded successfully")

# ── 4. Preview ────────────────────────────────────────────────────────────
with engine.connect() as conn:
    preview = pd.read_sql(
        "SELECT * FROM cyberbullying_statistics_2024 LIMIT 10",
        conn
    )

print("\nPreview:")
print(preview.to_string(index=False, formatters={
    "Value": "{:,.2f}".format
}))


## 📈 Cyberbullying Statistics 2024 – Visualization Dashboard (Live from RDS)

This section connects to the `shieldspace` AWS MySQL RDS database and loads the `cyberbullying_statistics_2024` table. After cleaning and transforming the dataset, it generates a suite of insightful visualizations focused on cyberbullying trends among teens in Australia.

### 📊 Visuals Included:
- **Cyberbullying Surge (2019 vs 2024):** Bar chart showing the spike in reports to eSafety.
- **Gender Split:** Pie chart revealing girls are twice as likely to be targeted.
- **Platform-wise Bullying:** Horizontal bar chart showing which social media platforms are most reported.
- **Response to Bullying:** Pie chart illustrating whether teens told someone, blocked the bully, or stayed silent.
- **Motivation Factors:** Bar chart showing bullying linked to gender, race, and sexuality.
- **Awareness vs Reporting:** Comparison of awareness levels vs actual school reporting.
- **Emotional Impact:** Bar chart of negative outcomes like low self-esteem, isolation, and helplessness.

This dashboard transforms raw RDS data into clear, student-friendly visuals to support cyberbullying awareness and education campaigns.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

# ── 1. Connect to MySQL RDS ─────────────────────────────────────────────
username = "admin"
password = "fit5120ta30"
host     = "shieldspace.ck1sygqoe3hd.us-east-1.rds.amazonaws.com"
database = "*********"

engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}/{database}")
df = pd.read_sql_table("cyberbullying_statistics_2024", con=engine)
print("✅ Data loaded from RDS")

# ── 2. Clean the data ───────────────────────────────────────────────────
df["Statistic"] = df["Statistic"].astype(str).str.strip().str.lower()
df["Category"] = df["Category"].astype(str).str.strip()
df["Value"] = pd.to_numeric(df["Value"], errors="coerce")
df.dropna(subset=["Value"], inplace=True)

# ── 3. Visualization Functions ──────────────────────────────────────────

def plot_cyberbullying_surge():
    years = [2019, 2024]
    reports = [536, 2978]
    plt.figure(figsize=(8, 5))
    bars = plt.bar(years, reports, color=["skyblue", "coral"])
    plt.text(2024, 2978 + 100, "12–13 yr: 35%", ha="center", fontsize=10, color="darkred")
    plt.title("Cyberbullying Reports to eSafety (2019 vs 2024)")
    plt.ylabel("Number of Reports")
    plt.xticks(years)
    plt.grid(axis="y", linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.show()

def plot_gender_split():
    plt.figure(figsize=(6, 6))
    plt.pie([2, 1], labels=["Girls", "Boys"], autopct="%1.1f%%", startangle=90, colors=["pink", "lightblue"])
    plt.title("Gender-Based Cyberbullying (12–13 yr olds): Girls 2x Boys")
    plt.tight_layout()
    plt.show()

def plot_platform_bullying():
    platform_df = df[df["Statistic"].str.contains("bullied on", na=False)]
    platforms = platform_df["Statistic"].str.extract(r"on (\w+)", expand=False).fillna("Unknown")
    values = platform_df["Value"]

    plt.figure(figsize=(8, 5))
    plt.barh(platforms, values, color=["#FF0000", "#FFFC00", "#69C9D0"])
    plt.xlabel("Percent of Users Bullied")
    plt.title("Bullying Reports on Social Media Platforms (Ages 10–18)")
    plt.grid(axis="x", linestyle="--", alpha=0.5)
    plt.xlim(0, 100)
    plt.tight_layout()
    plt.show()

def plot_silence_response():
    told = df[df["Statistic"].str.contains("told someone", na=False)]["Value"].values[0]
    blocked = df[df["Statistic"].str.contains("blocked", na=False)]["Value"].values[0]
    did_not_tell = 100 - told

    labels = ["Told Someone", "Blocked Bully", "Did Not Tell"]
    values = [told, blocked, did_not_tell]
    colors = ["#90ee90", "#ffcccb", "#d3d3d3"]

    plt.figure(figsize=(6, 6))
    plt.pie(values, labels=labels, autopct="%1.0f%%", startangle=140, colors=colors)
    plt.title("Teen Reactions to Cyberbullying")
    plt.tight_layout()
    plt.show()

def plot_motivation_venn():
    motivation_df = df[df["Statistic"].str.contains("motivated|based", na=False)]
    if motivation_df.empty:
        print("❌ No motivation data found.")
        return

    label_map = {
        "gender-based offensive content": "Gender",
        "race-based offensive content": "Race",
        "sexuality-based offensive content": "Sexuality",
        "racially motivated cyberbullying": "Race",
        "gender-based cyberbullying": "Gender",
        "sexuality-based cyberbullying": "Sexuality"
    }

    motivation_df["Label"] = motivation_df["Statistic"].map(label_map).fillna("Other")
    grouped = motivation_df.groupby("Label")["Value"].sum()

    plt.figure(figsize=(7, 5))
    plt.bar(grouped.index, grouped.values, color=["plum", "lightgreen", "lightskyblue"])
    plt.ylabel("Percent of Cases")
    plt.title("Motivations Behind Cyberbullying")
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

def plot_awareness_vs_reporting():
    awareness = df[df["Statistic"].str.contains("aware of cyberbullying", na=False)]["Value"].values[0]
    reported = df[df["Statistic"].str.contains("reported harassment to school", na=False)]["Value"].values[0]

    labels = ["Awareness", "Reported to School"]
    values = [awareness, reported]

    plt.figure(figsize=(6, 5))
    plt.bar(labels, values, color=["orange", "steelblue"])
    plt.title("Awareness vs Reporting Cyberbullying")
    plt.ylabel("Percent")
    plt.grid(axis="y", linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.show()

def plot_emotional_impact():
    impact_df = df[df["Statistic"].str.contains("self-esteem|isolated|helpless", na=False)]

    if impact_df.empty:
        print("❌ No emotional impact data found.")
        return

    label_map = {
        "teens developed self-esteem issues": "Self-esteem",
        "teens felt isolated": "Isolation",
        "teens felt helpless": "Helplessness",
        "38% of bullied teens developed self-esteem issues": "Self-esteem",
        "33% felt isolated": "Isolation",
        "20% felt helpless": "Helplessness"
    }

    impact_df["Label"] = impact_df["Statistic"].map(label_map).fillna("Other")
    grouped = impact_df.groupby("Label")["Value"].sum()

    plt.figure(figsize=(7, 5))
    plt.bar(grouped.index, grouped.values, color=["gray", "lightblue", "lightcoral"])
    plt.ylabel("Percent of Teens Affected")
    plt.title("Emotional Impact of Cyberbullying on Teens")
    plt.grid(axis="y", linestyle="--", alpha=0.5)
    plt.ylim(0, 50)
    plt.tight_layout()
    plt.show()

# ── 4. Run All Visualizations ────────────────────────────────────────────
plot_cyberbullying_surge()
plot_gender_split()
plot_platform_bullying()
plot_silence_response()
plot_motivation_venn()
plot_awareness_vs_reporting()
plot_emotional_impact()